In [1]:
from re import L
import numpy as np
import matplotlib as plt
from random import uniform as uf

#'modelo'
class Disco:
    def __init__(self, elradio, lamasa, laposicionx, laposiciony, lavelocidadx, lavelocidady):
        self.radio = elradio
        self.masa = lamasa
        self.posicionx = laposicionx
        self.posiciony = laposiciony
        self.velocidadx = lavelocidadx
        self.velocidady = lavelocidady
        #variacion de las fronteras
        self.top = laposiciony + elradio
        self.bottom = laposiciony - elradio
        self.left = laposicionx - elradio
        self.right = laposicionx + elradio
        #posicion de los discos en el eje X para crear el histograma
        self.arrayposicionx = np.array([self.posicionx])

class Box:
    def __init__(self,lalongitudX,lalongitudY):
      self.longitudx = lalongitudX
      self.longitudy = lalongitudY

class Grilla:
  def __init__(self,xmax,ldiscos):
    self.xmax = xmax

    def creacion_grilla(self,xmax,ldiscos):
      #creación de la grilla para detección
      subdivisionesx = int(xmax/(2*ldiscos[0].radio)) #cantidad de subdivisiones en el eje x
      divisionX = np.linspace(0,xmax,subdivisionesx)
      dist_entre_separX = divisionX[1]
      print(dist_entre_separX)
      return divisionX, dist_entre_separX
    self.divisionX = creacion_grilla(self,xmax,ldiscos)[0]
    self.dist_entre_separX = creacion_grilla(self,xmax,ldiscos)[1]

#'controlador'
def nueva_posicion(disco, dt):
    #funcion para una nueva posicion del disco al moverse
    #con la ec. x_f = x_0 + vt
    disco.posicionx += disco.velocidadx * dt
    disco.posiciony += disco.velocidady * dt
    disco.arrayposicionx = np.append(disco.arrayposicionx,disco.posicionx)

'''def tiempo_colision_pared(disco, lado):
  #funcion para obtener el tiempo que toma para chocar con
  #la primera pared (será el tiempo mínimo de la lista tiempos)
  tiempos = []
  for i in range(2): #i va a ser x o y para posicion/velocidad
    if disco.velocidad[i] > 0: #en este caso va para la pared en xmax o en ymax
      tiempos.append((lado-disco.radio - disco.posicion[i]) / disco.velocidad[i])
    elif disco.velocidad[i] < 0: #en este caso va para cero en x o y
      tiempos.append((disco.radio - disco.posicion[i]) / disco.velocidad[i])
  return min(tiempos)'''

def cambio_velocidad_colison_pares(disco1,disco2):
  disco1.velocidadx = (disco1.velocidadx*(disco1.masa - disco2.masa) + 2*disco2.masa*disco2.velocidadx)/(disco1.masa + disco2.masa)
  disco1.velocidady = (disco1.velocidady*(disco1.masa - disco2.masa) + 2*disco2.masa*disco2.velocidady)/(disco1.masa + disco2.masa)
  disco2.velocidadx = (disco2.velocidadx*(disco2.masa - disco1.masa) + 2*disco1.masa*disco1.velocidadx)/(disco1.masa + disco2.masa)
  disco2.velocidady = (disco2.velocidady*(disco2.masa - disco1.masa) + 2*disco1.masa*disco1.velocidady)/(disco1.masa + disco2.masa)

def deteccion_colision_pared(disco,lx,ly):
  #Esta parte viene de realizar una parametrización de la trayectoria del disco por medio de la ecuación paramétrica de la recta.
  #Buscamos el tiempo t entre el intervalo de [0,1] en el que se causa la colisión con la pared
  if disco.velocidadx > 0:
    posicionfinal = disco.right + disco.velocidadx
    if posicionfinal >= lx:
      t = (lx - disco.right)/(posicionfinal - disco.right)
      x = disco.right + t*(posicionfinal-disco.right)
      disco.posicionx = x - disco.radio
      disco.velocidadx = disco.velocidadx * -1
      print("Hay colision Pared der")

  if disco.velocidadx < 0:
    posicionfinal = disco.left + disco.velocidadx
    if posicionfinal <= 0:
      t = disco.left/(posicionfinal - disco.left)
      x = disco.left + t*(posicionfinal-disco.left)
      disco.posicionx = x + disco.radio
      disco.velocidadx = disco.velocidadx * -1
      print("Hay colision Pared izq")

  if disco.velocidady > 0:
    posicionfinal = disco.top + disco.velocidady
    if posicionfinal >= ly:
      t = (ly - disco.top)/(posicionfinal - disco.top)
      y = disco.top + t*(posicionfinal-disco.top)
      disco.posiciony = y - disco.radio
      disco.velocidady = disco.velocidady * -1
      print("Hay colision arr")

  if disco.velocidady < 0:
    posicionfinal = disco.bottom + disco.velocidady
    if posicionfinal <= 0:
      t = disco.bottom/(posicionfinal - disco.bottom)
      y = disco.bottom + t*(posicionfinal-disco.bottom)
      disco.posiciony = y + disco.radio
      disco.velocidady = disco.velocidady * -1
      print("Hay colision aba")

def deteccion_colision_pares(grilla,ldiscos):
  #función para el cambio en la velocidad de los discos que colisionan
  def cambio_velocidad_colision_pares(disco1,disco2):
    disco1.velocidadx = (disco1.velocidadx*(disco1.masa - disco2.masa) + 2*disco2.masa*disco2.velocidadx)/(disco1.masa + disco2.masa)
    disco1.velocidady = (disco1.velocidady*(disco1.masa - disco2.masa) + 2*disco2.masa*disco2.velocidady)/(disco1.masa + disco2.masa)
    disco2.velocidadx = (disco2.velocidadx*(disco2.masa - disco1.masa) + 2*disco1.masa*disco1.velocidadx)/(disco1.masa + disco2.masa)
    disco2.velocidady = (disco2.velocidady*(disco2.masa - disco1.masa) + 2*disco1.masa*disco1.velocidady)/(disco1.masa + disco2.masa)


  ################################################## Borrar el codigo marcado entre comillas triples ########################################################
  """#verificación de posiciones en la grilla
  contador = 0
  for j in divisionX:
    discos_en_grilla = []
    for i in range(len(ldiscos)):
      if ldiscos[i].posicionx - j < dist_entre_separX:
          discos_en_grilla.append(ldiscos[i])
    #descartamos los discos que no se sobreponen en X
    for i in discos_en_grilla:
      for k in discos_en_grilla:
        if i == k:
          pass
        else:
          if abs(i.right - k.left) <= abs(k.posicionx - i.posicionx) or (i.left - k.right) <= abs(k.posicionx - i.posicionx):
            #hay posible colisión
            if abs(i.top - k.bottom) <= abs(k.posiciony - i.posiciony) or (i.bottom - k.top) <= abs(k.posiciony - i.posiciony):
              contador += 1
                #hay colisión confirmada
              cambio_velocidad_colision_pares(i,k)
      discos_en_grilla.remove(i) #Se elimina el disco con el que se comprobó si había colisiones.

  print(contador)"""

  ########################################## Mejora del sistema de grilla ##########################################################
  """


  Borrar el contador de colisiones cuando ya esté todo listo


  """
  contador = 0
  #verificación de posiciones en un grilla y en sus vecinas inmediatas sobre el eje x
  for j in grilla.divisionX:
    discos_en_grilla = []
    for i in range(len(ldiscos)):
      if abs(ldiscos[i].posicionx - j) < (2 * grilla.dist_entre_separX):
          discos_en_grilla.append(ldiscos[i])
    #descartamos los discos que no se sobreponen en X
    for i in discos_en_grilla:
      for k in discos_en_grilla:
        if i == k:
          pass
        else:
          if abs(i.right - k.left) <= abs(k.posicionx - i.posicionx) or (i.left - k.right) <= abs(k.posicionx - i.posicionx):
            #hay posible colisión
            if abs(i.top - k.bottom) <= abs(k.posiciony - i.posiciony) or (i.bottom - k.top) <= abs(k.posiciony - i.posiciony):
              contador += 1
                #hay colisión confirmada
              cambio_velocidad_colision_pares(i,k)
      discos_en_grilla.remove(i) #Se elimina el disco con el que se comprobó si había colisiones.
  msj = "numero de colisiones: " + str(contador)
  print(msj)

def acomodo_inicial_discos(radio, masa, velMin, velMax, caja, num_discos): #esta es una función para el caso en el que se quiera hacer un acomodo de posiciones para un numero por definir de discos
  ldiscos = []
  if (num_discos/2)%(2) == 0:
    num_subdivisionx = int(num_discos/2)
    num_subdivisiony = num_subdivisionx
    posiciones_x = np.linspace(radio,caja.longitudx-radio,num_subdivisionx)
    posiciones_y = np.linspace(radio,caja.longitudy-radio,num_subdivisiony)
  else:
    num_subdivisionx = int((num_discos+1)/2)
    num_subdivisiony = int((num_discos-1)/2)
    posiciones_x = np.linspace(radio,caja.longitudx-radio,num_subdivisionx)
    posiciones_y = np.linspace(radio,caja.longitudy-radio,num_subdivisiony)
  for i in range(num_subdivisionx):
    for j in range(num_subdivisiony):
      posX = posiciones_x[i]
      posY = posiciones_y[j]
      veloX = uf(velMin, velMax)
      veloY = uf(velMin, velMax)
      ldiscos.append(Disco(radio, masa, posiciones_x[i], posiciones_y[j] ,veloX, veloY))
    np.append(ldiscos[i].arrayposicionx,ldiscos[i].posicionx)

  return ldiscos

#'vista'

def main():
    ################################### Parámetros iniciales del sistema ###################################
    #dimension de la caja
    ladohorizontal = 1
    ladovertical = 1
    caja = Box(ladohorizontal,ladovertical)
    #incializacion de discos
    """discos = []"""
    numero_discos = 4
    masa = 1
    radio = 0.05

    ############## Esta función sirve para inicializar los discos previendo que no se sobrepongan al crearse, y está abierto a cualquier número de discos ##############
    discos = acomodo_inicial_discos(radio, masa, -0.1, 0.1, caja, numero_discos)

    ############## Este código marcado entre comillas triples es para la inicialización de los discos en posiciones establecidas, pero depende de hacerlo a mano ##############
    """############inicializacion de los discos por medio de unas posicones conocidas pero con velocidades aleatorias##################################
    posicionX = [0.1, 0.1, 0.9, 0.9]
    posicionY = [0.1, 0.9, 0.1, 0.9]
    velocidadX = [uf(-0.10,0.10),uf(-0.10,0.10),uf(-0.10,0.10),uf(-0.10,0.10)]
    velocidadY = [uf(-0.10,0.10),uf(-0.10,0.10),uf(-0.10,0.10),uf(-0.10,0.10)]
    for i in range(numero_discos):
      discos.append(Disco(radio, masa, posicionX[i], posicionY[i] ,velocidadX[i], velocidadY[i]))
    ################################################################################################################################################"""
    """Creacion del obejeto grilla para la detección de colisiones"""

    grilla = Grilla(caja.longitudx,discos)
    #evolucion temporal
    FPS = 30
    newt = 1/FPS
    tmax = 4
    tprima = 0
    timearray = []
    for n in range(0,tmax*FPS+1):
      #actualizacion de posiciones
      #tprima es el tiempo actual del sistema
      tprima = n*newt
      """pasar a un numpy array el timearray"""
      timearray.append(tprima)
      #actualizacion de las posiciones de los discos
      for i in range(numero_discos):
        nueva_posicion(discos[i],tprima)
        #verificación de colisiones con las paredes
        deteccion_colision_pared(discos[i],caja.longitudx,caja.longitudy)
      #verificacion de colisiones entre los discos
      deteccion_colision_pares(grilla,discos)
main()

0.1111111111111111
0.1111111111111111
Hay colision Pared izq
Hay colision aba
Hay colision Pared izq
Hay colision arr
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay colision aba
numero de colisiones: 7
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 8
Hay colision aba
Hay colision aba
numero de colisiones: 10
Hay colision Pared izq
Hay colision aba
Hay colision aba
numero de colisiones: 6
Hay colision aba
Hay c